In [32]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import matplotlib.cm as cm
import matplotlib.colors as colors
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import folium

print('Done!')

Done!


### PART 1 ###

In [35]:
from bs4 import BeautifulSoup
url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data = requests.get(url).text
soup = BeautifulSoup(data, 'html5lib')

In [36]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
toronto=pd.DataFrame(table_contents)
toronto['Borough']=toronto['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [38]:
print('Boroughs {} Neighborhoods {}'.format(
        len(toronto['Borough'].unique()),
        toronto.shape[0]
    )
)

Boroughs 15 Neighborhoods 103


### PART 2 ###

In [39]:
file='Geospatial_Coordinates.csv'
df=pd.read_csv(file)
df.sort_values(by=["Postal Code"])
df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [41]:
neighborhoods = pd.DataFrame(columns=['Borough', 'Neighborhood', 'Latitude', 'Longitude'] )
for i in range(0,len(df)):
    key = df["Postal Code"][i]
    values = toronto[toronto['PostalCode'] == key]
    borough = values["Borough"].values[0]
    Neighborhood = values["Neighborhood"].values[0]
    neighborhoods = neighborhoods.append({'PostCode': df["Postal Code"][i],
                                      'Borough': borough,
                                      'Neighborhood': Neighborhood,
                                      'Latitude': df["Latitude"][i],
                                      'Longitude': df["Longitude"][i]}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude,PostCode
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,M1B
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,M1C
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,M1E
3,Scarborough,Woburn,43.770992,-79.216917,M1G
4,Scarborough,Cedarbrae,43.773136,-79.239476,M1H


### PART 3 ###

In [42]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto Latitude {}, Toronto Longitude {}.'.format(latitude, longitude))

Toronto Latitude 43.6534817, Toronto Longitude -79.3839347.


In [46]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [47]:
CLIENT_ID = 'LCNYC5QNHSU4HUNAGOPQZ21PVL1ADJ3D5W5O45OKTO1MO1LP'
CLIENT_SECRET = 'HBOZN2YZJARA2ZDF1F4CB3GWC4PRPXSQIJ2CWQCJLGKNC0C3'
ACCESS_TOKEN = ''
VERSION = '20180604'
LIMIT = 30
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

CLIENT_ID: LCNYC5QNHSU4HUNAGOPQZ21PVL1ADJ3D5W5O45OKTO1MO1LP
CLIENT_SECRET:HBOZN2YZJARA2ZDF1F4CB3GWC4PRPXSQIJ2CWQCJLGKNC0C3


In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
torontoVenues = getNearbyVenues(neighborhoods["Neighborhood"],neighborhoods["Latitude"],neighborhoods["Longitude"])

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Don Mills South
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
The Danforth  East
The Danforth West, Riverdale


In [52]:
print(torontoVenues.shape)
torontoVenues.head()

(1327, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern, Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Great Shine Window Cleaning,43.783145,-79.157431,Home Service
2,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping
4,"Guildwood, Morningside, West Hill",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank


In [54]:
print('Uniques categories {}'.format(len(torontoVenues['Venue Category'].unique())))
torontoVenues.groupby('Neighborhood').count()

Uniques categories 232


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",9,9,9,9,9,9
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",23,23,23,23,23,23
Berczy Park,30,30,30,30,30,30
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16


In [55]:
toronto_onehot = pd.get_dummies(torontoVenues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood'] = torontoVenues['Neighborhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
print(toronto_onehot.shape)
toronto_onehot.head()

(1327, 232)


,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Cafeteria,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Drugstore,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Organic Grocery,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"Malvern, Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [56]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(100, 232)

In [79]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]


In [80]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Skating Rink,Breakfast Spot,Latin American Restaurant,Women's Store,Curling Ice,Dog Run,Distribution Center,Discount Store,Diner
1,"Alderwood, Long Branch",Pizza Place,Athletics & Sports,Pharmacy,Skating Rink,Sandwich Place,Pub,Coffee Shop,Gym,Creperie,College Auditorium
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Sushi Restaurant,Shopping Mall,Sandwich Place,Bridal Shop,Restaurant,Pizza Place,Pharmacy,Park
3,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Restaurant,Italian Restaurant,Greek Restaurant,Thai Restaurant,Liquor Store,Juice Bar,Indian Restaurant,Pub


In [81]:
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 0], dtype=int32)

In [82]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude,PostCode
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,M1B
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,M1C
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,M1E
3,Scarborough,Woburn,43.770992,-79.216917,M1G
4,Scarborough,Cedarbrae,43.773136,-79.239476,M1H


In [83]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = neighborhoods
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [84]:
toronto_merged.dropna(subset=["Cluster Labels"], inplace=True)
toronto_merged

,Borough,Neighborhood,Latitude,Longitude,PostCode,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,"Malvern, Rouge",43.806686,-79.194353,M1B,2.0,Fast Food Restaurant,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497,M1C,2.0,Bar,Construction & Landscaping,Home Service,Women's Store,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711,M1E,2.0,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Bank,Medical Center,Intersection,Restaurant,Donut Shop,Construction & Landscaping
3,Scarborough,Woburn,43.770992,-79.216917,M1G,2.0,Coffee Shop,Korean BBQ Restaurant,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
4,Scarborough,Cedarbrae,43.773136,-79.239476,M1H,2.0,Bank,Athletics & Sports,Bakery,Hakka Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Women's Store,Deli / Bodega
5,Scarborough,Scarborough Village,43.744734,-79.239476,M1J,3.0,Playground,Women's Store,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
6,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029,M1K,2.0,Chinese Restaurant,Coffee Shop,Convenience Store,Department Store,Hobby Shop,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Women's Store
7,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577,M1L,2.0,Bakery,Bus Line,Park,Bus Station,Intersection,Ice Cream Shop,Metro Station,Soccer Field,Dance Studio,Dog Run
8,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476,M1M,2.0,Skating Rink,American Restaurant,Motel,Women's Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
9,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848,M1N,2.0,General Entertainment,Skating Rink,College Stadium,Café,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner


In [85]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[round(cluster)-1],
        fill=True,
        fill_color=rainbow[round(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [86]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,"Milliken, Agincourt North, Steeles East, L'Amo...",0.0,Coffee Shop,Intersection,Playground,Park,Gay Bar,Cosmetics Shop,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
21,"Willowdale, Newtonbrook",0.0,Park,Women's Store,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
23,York Mills West,0.0,Park,Construction & Landscaping,Convenience Store,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
25,Parkwoods,0.0,Park,Food & Drink Shop,Women's Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
30,Downsview East,0.0,Park,Business Service,Airport,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
40,The Danforth East,0.0,Coffee Shop,Park,Intersection,Convenience Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
44,Lawrence Park,0.0,Park,Bus Line,Construction & Landscaping,Swim School,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
48,"Moore Park, Summerhill East",0.0,Restaurant,Playground,Park,College Cafeteria,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
50,Rosedale,0.0,Park,Playground,Trail,Coworking Space,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
64,Forest Hill North & West,0.0,Park,Jewelry Store,Trail,Sushi Restaurant,Women's Store,Creperie,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


In [87]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
32,Downsview Central,1.0,Baseball Field,Home Service,Food Truck,Women's Store,Curling Ice,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner
91,"Old Mill South, King's Mill Park, Sunnylea, Hu...",1.0,Business Service,Baseball Field,Construction & Landscaping,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
97,"Humberlea, Emery",1.0,Baseball Field,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [88]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Malvern, Rouge",2.0,Fast Food Restaurant,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
1,"Rouge Hill, Port Union, Highland Creek",2.0,Bar,Construction & Landscaping,Home Service,Women's Store,Curling Ice,Drugstore,Donut Shop,Dog Run,Distribution Center,Discount Store
2,"Guildwood, Morningside, West Hill",2.0,Rental Car Location,Mexican Restaurant,Electronics Store,Breakfast Spot,Bank,Medical Center,Intersection,Restaurant,Donut Shop,Construction & Landscaping
3,Woburn,2.0,Coffee Shop,Korean BBQ Restaurant,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
4,Cedarbrae,2.0,Bank,Athletics & Sports,Bakery,Hakka Restaurant,Thai Restaurant,Gas Station,Fried Chicken Joint,Caribbean Restaurant,Women's Store,Deli / Bodega
6,"Kennedy Park, Ionview, East Birchmount Park",2.0,Chinese Restaurant,Coffee Shop,Convenience Store,Department Store,Hobby Shop,Dim Sum Restaurant,Dance Studio,Deli / Bodega,Dessert Shop,Women's Store
7,"Golden Mile, Clairlea, Oakridge",2.0,Bakery,Bus Line,Park,Bus Station,Intersection,Ice Cream Shop,Metro Station,Soccer Field,Dance Studio,Dog Run
8,"Cliffside, Cliffcrest, Scarborough Village West",2.0,Skating Rink,American Restaurant,Motel,Women's Store,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant
9,"Birch Cliff, Cliffside West",2.0,General Entertainment,Skating Rink,College Stadium,Café,Women's Store,Cuban Restaurant,Dog Run,Distribution Center,Discount Store,Diner
10,"Dorset Park, Wexford Heights, Scarborough Town...",2.0,Indian Restaurant,Pet Store,Vietnamese Restaurant,Chinese Restaurant,Creperie,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant


In [89]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Scarborough Village,3.0,Playground,Women's Store,Creperie,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop


In [90]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,"West Deane Park, Princess Gardens, Martin Grov...",4.0,Bakery,Women's Store,Cuban Restaurant,Donut Shop,Dog Run,Distribution Center,Discount Store,Diner,Dim Sum Restaurant,Dessert Shop
